In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from copy import deepcopy
import random

import tensorflow as tf
import keras


# %matplotlib notebook
%matplotlib inline

In [6]:
with open('cloud_examples/airplane_no_noise/P.obj') as in_file:
    raw_data = in_file.read()
    
dataP = [list(map(float, row.split(' ')[1:])) for row in raw_data.split('\n')][:-1]

with open('cloud_examples/airplane_no_noise/Q.obj') as in_file:
    raw_data = in_file.read()
    
dataQ = [list(map(float, row.split(' ')[1:])) for row in raw_data.split('\n')][:-1]

In [7]:
Q = np.array(dataQ)
P = np.array(dataP)

## Ищем дисперсию и обратное средннеквадратичное отклонение

In [33]:
p_mean = np.array([P[:,0].mean(), P[:,1].mean(), P[:,2].mean()])

In [49]:
n = P.shape[0]
n

1024

In [58]:
sigma = np.array([np.sqrt(np.sum((P[:, i] - p_mean[i])**2) /n) for i in range(0, 3)])
sigma

array([0.24251311, 0.46149122, 0.0741362 ])

In [57]:
b = 1/sigma
b

array([ 4.12348835,  2.16688845, 13.48868751])

## Строим слой сети

In [59]:
from keras.layers import Layer
from keras import backend as K

In [165]:
class RBFLayer(keras.layers.Layer):
    def __init__(self, input_dim=3, point_count=1024):
        super().__init__()
        self.w = self.add_weight(
            shape=(point_count, input_dim), initializer="random_normal", trainable=True
        )

    def call(self, p):
        """
        inputs = p
        """
        self.culc_b(p)
        diff = p - self.w
        l2 = tf.sqrt(tf.reduce_sum(tf.pow(diff, 2), axis=1))
        n1 = l2 * self.b
        return tf.reshape(tf.exp(-tf.pow(n1, 2)), (1, 1024))
    
    def culc_b(self, inputs):
        means = tf.reduce_mean(inputs, axis=0)
        n = inputs.shape[0]
        self.b = tf.sqrt(tf.reduce_sum(tf.pow(tf.pow(tf.sqrt(tf.reduce_sum(tf.pow(inputs - means, 2), axis=0)/n), -1), 2), axis=0))
        

In [432]:
class RBFLayerForQ(keras.layers.Layer):
    def __init__(self, q):
        super().__init__()
        self.w = q

    def call(self, p):
        """
        inputs = p
        """
        self.culc_b(p)
        p = tf.cast(p, tf.float64)
        diff = p - self.w
        l2 = tf.sqrt(tf.reduce_sum(tf.pow(diff, 2), axis=1))
        n1 = l2 * self.b
        return tf.reshape(tf.exp(-tf.pow(n1, 2)), (1, 1024))
    
    def culc_b(self, inputs):
        means = tf.reduce_mean(inputs, axis=0)
        n = inputs.shape[0]
        b = tf.sqrt(tf.reduce_sum(tf.pow(tf.pow(tf.sqrt(tf.reduce_sum(tf.pow(inputs - means, 2), axis=0)/n), -1), 2), axis=0))
        self.b = tf.cast(b, tf.float64)

In [433]:
class CompetitiveLayer(keras.layers.Layer):
    def __init__(self, input_dim=1, point_count=1024):
        super().__init__()
        self.w = self.add_weight(
            shape=(point_count, point_count), initializer="random_normal", trainable=True
        )

    def call(self, a1):
        return tf.nn.relu(tf.matmul(a1, self.w))

In [435]:
rbf_layer = RBFLayerForQ(q)
y = rbf_layer(p)
y

<tf.Tensor: shape=(1, 1024), dtype=float64, numpy=
array([[3.98177406e-38, 2.26444306e-23, 4.96082477e-33, ...,
        2.93109074e-26, 1.04523635e-22, 5.23556013e-23]])>

In [436]:
com_layer = CompetitiveLayer(1, 1024)

In [437]:
test = com_layer(y)
test

<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[0.000000e+00, 0.000000e+00, 9.588542e-20, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00]], dtype=float32)>

In [495]:
test = com_layer(test)
len([i.numpy() for i in test[0] if i.numpy() == 0])

511

In [494]:
for i in range(100):
    test = com_layer(test)

test

<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[6.8420465e+09, 4.0962703e+10, 7.2626719e+09, ..., 0.0000000e+00,
        0.0000000e+00, 2.4148230e+10]], dtype=float32)>

## Пробуем что-то с слоями

In [362]:
from keras import models
from keras import layers

In [363]:
model = models.Sequential()
model.add(RBFLayer(3, 1024))
model.add(CompetitiveLayer(1, 1024))

## Sandbox

In [395]:
q = tf.constant(Q)
q

<tf.Tensor: shape=(1024, 3), dtype=float64, numpy=
array([[-0.552865  ,  0.24598   ,  0.280693  ],
       [-0.316073  ,  0.0695407 , -0.00655038],
       [-0.630708  ,  0.200029  ,  0.19348   ],
       ...,
       [-0.324171  , -0.00459492,  0.066088  ],
       [-0.582059  , -0.0138649 , -0.074342  ],
       [-0.454034  , -0.370308  , -0.318585  ]])>

In [284]:
p = tf.constant(P, dtype='float64')
p

<tf.Tensor: shape=(1024, 3), dtype=float64, numpy=
array([[ 0.056,  0.367,  0.087],
       [ 0.174,  0.036, -0.128],
       [-0.045,  0.293,  0.077],
       ...,
       [ 0.204,  0.013, -0.031],
       [-0.086, -0.034, -0.029],
       [-0.03 , -0.479, -0.073]])>

In [287]:
p - q

<tf.Tensor: shape=(1024, 3), dtype=float64, numpy=
array([[ 0.608865  ,  0.12102   , -0.193693  ],
       [ 0.490073  , -0.0335407 , -0.12144962],
       [ 0.585708  ,  0.092971  , -0.11648   ],
       ...,
       [ 0.528171  ,  0.01759492, -0.097088  ],
       [ 0.496059  , -0.0201351 ,  0.045342  ],
       [ 0.424034  , -0.108692  ,  0.245585  ]])>

In [420]:
qwer = tf.constant([[2]], dtype='double')

In [421]:
q * qwer

<tf.Tensor: shape=(1024, 3), dtype=float64, numpy=
array([[-1.10573   ,  0.49196   ,  0.561386  ],
       [-0.632146  ,  0.1390814 , -0.01310076],
       [-1.261416  ,  0.400058  ,  0.38696   ],
       ...,
       [-0.648342  , -0.00918984,  0.132176  ],
       [-1.164118  , -0.0277298 , -0.148684  ],
       [-0.908068  , -0.740616  , -0.63717   ]])>